In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 748.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
import torch
import plotly.graph_objs as go
from torch_geometric.data import Data
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, Dataset, DataLoader
from torch_geometric.nn import GCNConv
import pandas as pd
import numpy as np


In [ ]:

# Define joints and edges
joints = [
    'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST', 'NECK', 'CLAVICLE_LEFT', 'SHOULDER_LEFT',
    'ELBOW_LEFT', 'WRIST_LEFT', 'HAND_LEFT', 'HANDTIP_LEFT', 'THUMB_LEFT',
    'CLAVICLE_RIGHT', 'SHOULDER_RIGHT', 'ELBOW_RIGHT', 'WRIST_RIGHT', 'HAND_RIGHT',
    'HANDTIP_RIGHT', 'THUMB_RIGHT', 'HIP_LEFT', 'KNEE_LEFT', 'ANKLE_LEFT',
    'FOOT_LEFT', 'HIP_RIGHT', 'KNEE_RIGHT', 'ANKLE_RIGHT', 'FOOT_RIGHT',
    'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT'
]

edges = [
    ('PELVIS', 'SPINE_NAVAL'), ('SPINE_NAVAL', 'SPINE_CHEST'), ('SPINE_CHEST', 'NECK'),
    ('NECK', 'HEAD'), ('SPINE_CHEST', 'CLAVICLE_LEFT'), ('CLAVICLE_LEFT', 'SHOULDER_LEFT'),
    ('SHOULDER_LEFT', 'ELBOW_LEFT'), ('ELBOW_LEFT', 'WRIST_LEFT'), ('WRIST_LEFT', 'HAND_LEFT'),
    ('HAND_LEFT', 'HANDTIP_LEFT'), ('WRIST_LEFT', 'THUMB_LEFT'), ('SPINE_CHEST', 'CLAVICLE_RIGHT'),
    ('CLAVICLE_RIGHT', 'SHOULDER_RIGHT'), ('SHOULDER_RIGHT', 'ELBOW_RIGHT'), ('ELBOW_RIGHT', 'WRIST_RIGHT'),
    ('WRIST_RIGHT', 'HAND_RIGHT'), ('HAND_RIGHT', 'HANDTIP_RIGHT'), ('WRIST_RIGHT', 'THUMB_RIGHT'),
    ('PELVIS', 'HIP_LEFT'), ('HIP_LEFT', 'KNEE_LEFT'), ('KNEE_LEFT', 'ANKLE_LEFT'),
    ('ANKLE_LEFT', 'FOOT_LEFT'), ('PELVIS', 'HIP_RIGHT'), ('HIP_RIGHT', 'KNEE_RIGHT'),
    ('KNEE_RIGHT', 'ANKLE_RIGHT'), ('ANKLE_RIGHT', 'FOOT_RIGHT'),
    ('HEAD', 'NOSE'),('HEAD','EYE_LEFT'), ('HEAD', 'EYE_RIGHT'), ('HEAD', 'EAR_LEFT'), ('HEAD', 'EAR_RIGHT')
]

joint_to_idx = {joint: idx for idx, joint in enumerate(joints)}

edge_index = torch.tensor(
    [[joint_to_idx[src], joint_to_idx[dst]] for src, dst in edges] +
    [[joint_to_idx[dst], joint_to_idx[src]] for src, dst in edges],  # Bidirectional edges
    dtype=torch.long
).t()

# Custom Dataset Class
class SkeletonDataset(Dataset):
    def __init__(self, csv_file):
        self.data = csv_file
        self.num_nodes = len(joints)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        # Extract node features and include timestamp
        node_features = []
        for joint in joints:
            x = row[f'{joint}_X']
            y = row[f'{joint}_Y']
            z = row[f'{joint}_Z']
            timestamp = row['timeStamps']  # Include timestamp as a feature
            node_features.append([x, y, z, timestamp])

        x = torch.tensor(node_features, dtype=torch.float)  # Shape: [num_nodes, 4]

        # Label
        y = torch.tensor(row['frailty_class'], dtype=torch.float)

        # Create data object
        data = Data(x=x, edge_index=edge_index, y=y)

        return data


In [ ]:

data_path = '/content/drive/MyDrive/Iqram Sir/AI_Frailty-MATLAB_Sample/Final_data_Balanced.csv'

dataframe = pd.read_csv(data_path)





# Load dataset
dataset = SkeletonDataset(csv_file=dataframe[dataframe['walking_speed'] == "Fast"])

# Split into training and testing datasets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
# Initialize Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HybridSTGCN(in_channels=4, hidden_channels=64, num_classes=1).to(device)

In [ ]:
# Define Loss Function and Optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader, Dataset
import itertools
from sklearn.metrics import accuracy_score

# Define the HybridSTGCN Model
class HybridSTGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_classes, dropout_rate=0.0):
        super(HybridSTGCN, self).__init__()

        # Spatial Graph Convolution Layers
        self.gcn1 = GCNConv(in_channels, hidden_channels)
        self.gcn2 = GCNConv(hidden_channels, hidden_channels)

        # Temporal Convolution Layers
        self.temporal_conv1 = nn.Conv1d(in_channels=hidden_channels,
                                        out_channels=hidden_channels,
                                        kernel_size=3, padding=1)

        # LSTM for temporal modeling
        self.lstm = nn.LSTM(hidden_channels, hidden_channels, batch_first=True)

        # Dropout layer
        self.dropout = nn.Dropout(p=dropout_rate)

        # Fully Connected Layer
        self.fc = nn.Linear(hidden_channels, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # Spatial GCN Layers
        x = F.relu(self.gcn1(x, edge_index))
        x = F.relu(self.gcn2(x, edge_index))

        # Reshape for Temporal Conv
        batch_size = batch.max().item() + 1
        num_nodes = x.size(0) // batch_size
        x = x.view(batch_size, num_nodes, -1)
        x = x.permute(0, 2, 1)

        # Temporal Conv Layer
        x = F.relu(self.temporal_conv1(x))

        # LSTM for capturing temporal patterns
        x = x.permute(0, 2, 1)
        x, _ = self.lstm(x)

        # Pooling over nodes
        x = x.mean(dim=1)

        # Apply Dropout
        x = self.dropout(x)

        # Fully Connected Layer
        x = self.fc(x)
        x = torch.sigmoid(x)  # For binary classification

        return x.squeeze()  # Ensure output is of shape [batch_size]

# Define the function to train the model
def train_model(model, train_loader, optimizer, criterion):
    model.train()
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y)  # Assuming data.y is of shape [batch_size]
        loss.backward()
        optimizer.step()

# Define the function to evaluate the model
def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data)

            # Ensure output is at least 1D
            if out.dim() == 0:
                out = out.unsqueeze(0)

            preds = (out >= 0.5).float()  # Binary classification threshold
            all_preds.extend(preds.cpu().numpy().tolist())  # Convert to list
            all_labels.extend(data.y.cpu().numpy().tolist())  # Convert to list

    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy

# Define hyperparameters for tuning
hyperparameters = {
    'learning_rate': [0.001, 0.0001],
    'hidden_channels': [32, 64, 128],
    'dropout_rate': [0.3, 0.5],
    'batch_size': [16, 32, 64]
}

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



# Initialize variables to store the best model and accuracy
best_accuracy = 0.0
best_params = {}

# Hyperparameter tuning loop
for lr, hidden_channels, dropout_rate, batch_size in itertools.product(
        hyperparameters['learning_rate'],
        hyperparameters['hidden_channels'],
        hyperparameters['dropout_rate'],
        hyperparameters['batch_size']):

    # Create the model with the current hyperparameters
    model = HybridSTGCN(in_channels=4, hidden_channels=hidden_channels, num_classes=1, dropout_rate=dropout_rate).to(device)

    # Adjust the DataLoader with the current batch size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Define the optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    # Train the model
    for epoch in range(10):  # Adjust the number of epochs as needed
        train_model(model, train_loader, optimizer, criterion)

    # Evaluate the model
    accuracy = evaluate_model(model, test_loader)
    print(f"Accuracy: {accuracy:.4f} | Params: lr={lr}, hidden_channels={hidden_channels}, dropout_rate={dropout_rate}, batch_size={batch_size}")

    # Save the best model and parameters
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = {
            'learning_rate': lr,
            'hidden_channels': hidden_channels,
            'dropout_rate': dropout_rate,
            'batch_size': batch_size
        }
        torch.save(model.state_dict(), 'best_model.pth')

print("Best Accuracy:", best_accuracy)
print("Best Parameters:", best_params)


Accuracy: 0.6876 | Params: lr=0.001, hidden_channels=32, dropout_rate=0.3, batch_size=16


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Accuracy: 0.7461 | Params: lr=0.001, hidden_channels=32, dropout_rate=0.3, batch_size=32


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Accuracy: 0.7299 | Params: lr=0.001, hidden_channels=32, dropout_rate=0.3, batch_size=64


...

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Accuracy: 0.8569 | Params: lr=0.0001, hidden_channels=128, dropout_rate=0.3, batch_size=16


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Accuracy: 0.7842 | Params: lr=0.0001, hidden_channels=128, dropout_rate=0.3, batch_size=32


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import numpy as np

def visualize_dataset(dataset):
    # Initialize the plotly figure
    fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])

    # Loop through each graph in the dataset to create frames
    frames = []
    for i, data in enumerate(dataset):
        node_positions = data.x[:, :3].numpy()  # Extract (x, y, z) coordinates, ignoring the timestamp

        # Extract x, y, z coordinates from node_positions
        x_coords = node_positions[:, 0]
        y_coords = node_positions[:, 1]
        z_coords = node_positions[:, 2]

        # Create edges for Plotly
        edge_x = []
        edge_y = []
        edge_z = []
        for edge in edge_index.t().tolist():
            x0, y0, z0 = node_positions[edge[0]]
            x1, y1, z1 = node_positions[edge[1]]
            edge_x.extend([x0, x1, None])
            edge_y.extend([y0, y1, None])
            edge_z.extend([z0, z1, None])

        # Plot the edges
        edge_trace = go.Scatter3d(
            x=edge_x, y=edge_y, z=edge_z,
            mode='lines',
            line=dict(color='black', width=2),
            hoverinfo='none'
        )

        # Plot the nodes with joint names
        node_trace = go.Scatter3d(
            x=x_coords, y=y_coords, z=z_coords,
            mode='markers+text',
            marker=dict(size=6, color='blue'),
            text=joints,  # Use the joint names as text labels
            hoverinfo='text'
        )

        # Add the frame
        frames.append(go.Frame(data=[edge_trace, node_trace], name=str(i)))

    # Create the initial plot with the first graph
    fig.add_trace(frames[0].data[0])
    fig.add_trace(frames[0].data[1])

    # Update the layout with sliders and animation settings
    fig.update_layout(
        title='3D Graph Animation of Dataset',
        scene=dict(
            xaxis=dict(showbackground=True, backgroundcolor="rgb(230, 230, 230)", gridcolor="rgb(200, 200, 200)", showgrid=True, zerolinecolor="rgb(200, 200, 200)"),
            yaxis=dict(showbackground=True, backgroundcolor="rgb(230, 230, 230)", gridcolor="rgb(200, 200, 200)", showgrid=True, zerolinecolor="rgb(200, 200, 200)"),
            zaxis=dict(showbackground=True, backgroundcolor="rgb(230, 230, 230)", gridcolor="rgb(200, 200, 200)", showgrid=True, zerolinecolor="rgb(200, 200, 200)")
        ),
        updatemenus=[dict(type='buttons', showactive=False,
                          buttons=[dict(label='Play',
                                        method='animate',
                                        args=[None, dict(frame=dict(duration=500, redraw=True), fromcurrent=True, mode='immediate')]),
                                   dict(label='Pause',
                                        method='animate',
                                        args=[[None], dict(frame=dict(duration=0, redraw=False), mode='immediate')])])],
        sliders=[{
            'steps': [{'args': [[f.name], dict(mode='immediate', frame=dict(duration=500, redraw=True))],
                       'label': str(k), 'method': 'animate'} for k, f in enumerate(frames)],
            'currentvalue': {'prefix': 'Frame: ', 'font': {'size': 20}},
            'pad': {'b': 10, 't': 50},
            'len': 0.9, 'x': 0.1, 'y': 0,
        }],
        width=1200, height=800, margin=dict(l=0, r=0, b=0, t=40)
    )

    # Add all frames to the figure
    fig.frames = frames

    fig.show()


visualize_dataset(dataset)



In [ ]:
import torch
from torch_geometric.nn import GNNExplainer
import matplotlib.pyplot as plt

# Assuming your model is defined as HybridSTGCN and trained
model = HybridSTGCN(in_channels=4, hidden_channels=64, num_classes=1)
model.load_state_dict(torch.load('model_checkpoint.pth'))  # Load your trained model
model.eval()  # Set the model to evaluation mode

# Initialize the GNNExplainer
explainer = GNNExplainer(model, epochs=200, return_type='prob')

# Explain a specific graph-level prediction
data = dataset[0]  # Use the first graph in the dataset (or any other graph)
node_feat_mask, edge_mask = explainer.explain_graph(data.x, data.edge_index)

# Visualize the explanation
ax, G = explainer.visualize_subgraph(node_idx=None, edge_index=data.edge_index, edge_mask=edge_mask, y=data.y)
plt.show()


ImportError: cannot import name 'GNNExplainer' from 'torch_geometric.nn' (/usr/local/lib/python3.10/dist-packages/torch_geometric/nn/__init__.py)

In [ ]:
!pip uninstall torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv

Found existing installation: torch_geometric 2.5.3
Uninstalling torch_geometric-2.5.3:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/torch_geometric-2.5.3.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torch_geometric/*
Proceed (Y/n)? Y
  Successfully uninstalled torch_geometric-2.5.3


In [ ]:
!pip install torch
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from torch_geometric.nn import GNNExplainer


ImportError: cannot import name 'GNNExplainer' from 'torch_geometric.nn' (/usr/local/lib/python3.10/dist-packages/torch_geometric/nn/__init__.py)

In [ ]:
pip install shap


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 9.8 MB/s eta 0:00:00


In [ ]:
import torch
from torch_geometric.nn import GCNConv
import shap

# Assume your model is already trained and defined as `model`
model = HybridSTGCN(in_channels=4, hidden_channels=64, num_classes=1)
model.eval()  # Set the model to evaluation mode


HybridSTGCN(
  (gcn1): GCNConv(4, 64)
  (gcn2): GCNConv(64, 64)
  (temporal_conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (lstm): LSTM(64, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)

In [ ]:
def gnn_predict(node_features_list):
    model.eval()
    preds = []
    for node_features in node_features_list:
        data = Data(x=torch.tensor(node_features, dtype=torch.float), edge_index=edge_index)
        data = data.to(device)
        with torch.no_grad():
            out = model(data)
            preds.append(out.item())  # Assuming binary classification
    return np.array(preds)


In [ ]:
# Convert the dataset into a list of node feature arrays (ignoring edge features for SHAP)
background = [data.x.numpy() for data in dataset[:100]]  # Use a subset of data for SHAP background


TypeError: Unknown type passed as data object: <class 'torch_geometric.data.data.Data'>